## 기본설정 및 함수정의

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objs as go
import plotly.offline as offline
from folium.plugins import HeatMapWithTime
from plotly.subplots import make_subplots
import folium
from folium import plugins
from folium.plugins import HeatMap
from folium import FeatureGroup
import json
import math
import re
from datetime import datetime
import os
import glob
import subprocess
from bs4 import BeautifulSoup as bs
from shapely.geometry import Point, Polygon, LineString
from shapely.ops import unary_union
import geopandas as gpd
from geopandas import GeoSeries
import pyproj
from tqdm import tqdm
from keplergl import KeplerGl

# tqdm의 pandas전용 메소드를 호출
tqdm.pandas()
# 모든 열이 생략되지 않도록 설정
pd.set_option('display.max_columns', None)

# 영등포역 위도, 경도
sejong = [36.51430420729354, 127.29034973889941 ]

# Point를만드는 함수
def make_point(x):
    try:
        return Point(x)
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
# Polygon을 만드는 함수
def make_pol(x):
    try:
        return Polygon(x[0])
    except:
        return Polygon(x[0][0])
    
# Linestring을 만드는 함수
def make_lin(x):
    try:
        return LineString(x)
    except:
        return LineString(x[0])

# 데이터프레임을 GeoPandas 데이터프레임으로 변환하는 함수 정의
def geo_transform(DataFrame) :
    # csv to geopandas
    # lon, lat data를 geometry로 변경
    DataFrame['lat'] = DataFrame['lat'].astype(float)
    DataFrame['lon'] = DataFrame['lon'].astype(float)
    DataFrame['geometry'] = DataFrame.progress_apply(lambda row : Point([row['lon'], row['lat']]), axis=1) # 위도 및 경도롤 GeoPandas Point 객체로 변환
    DataFrame = gpd.GeoDataFrame(DataFrame, geometry='geometry')
    DataFrame.crs = {'init':'epsg:4326'} # geopandas 데이터프레임의 좌표계를 EPSG 4326으로 설정
    DataFrame = DataFrame.to_crs({'init':'epsg:4326'}) # 데이터프레임의 좌표계를 자체 좌표계에서 EPSG 4326으로 변환
    return DataFrame

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\geopandas\_compat.py:124: UserWarning:

The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.

C:\Users\HJK\AppData\Local\Temp\ipykernel_27052\3172343792.py:23: DeprecationWarning:

Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using

#### 세종시 행정경계(대상구역)

In [2]:
# GeoJSON 파일 불러오기
with open('SBJ_2406_002/11.세종시_대상구역도.geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
object_area_df = pd.json_normalize(geojson_data['features'])
object_area_df['geometry'] = object_area_df['geometry.coordinates'].apply(lambda x : make_pol(x))
object_area_df.drop(columns="geometry.coordinates", inplace=True)
object_area_df.rename(columns = {'properties.EMD_NM' : 'ADM_NM'}, inplace = True)
object_area_df = object_area_df[['ADM_NM', 'geometry']]

# shp to geodataframe convert
shapefile_path = "SBJ_2406_002/_census_data_2023_bnd_dong_bnd_dong_29010_2023_2023"
sejong_gdf = gpd.read_file(shapefile_path)
sejong_gdf = sejong_gdf.to_crs(epsg=4326) #EPSG4326 형식으로 변환
sejong_gdf = sejong_gdf[['ADM_NM', 'geometry']]
sejong_gdf = sejong_gdf[~sejong_gdf['ADM_NM'].str.endswith('동')]
sejong_gdf =  pd.concat([object_area_df, sejong_gdf], ignore_index=True)
# sejong_gdf 데이터프레임을 GeoDataFrame으로 변환
sejong_gdf = gpd.GeoDataFrame(sejong_gdf, geometry='geometry')
# 조건에 따라 'newtown' 열 생성 및 값 지정
sejong_gdf['newtown'] = sejong_gdf['ADM_NM'].apply(lambda x: '신도시' if x.endswith('동') else '그외지역')

#### 격자(매핑용)

In [3]:
# GeoJSON 파일 불러오기
with open('SBJ_2406_002/4.세종시_격자(매핑용).geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
grid_map_df = pd.json_normalize(geojson_data['features'])
grid_map_df['geometry'] = grid_map_df['geometry.coordinates'].apply(lambda x : make_pol(x))
grid_map_df.drop(columns="geometry.coordinates", axis=1, inplace=True)
# grid_map_df 데이터프레임을 GeoDataFrame으로 변환
grid_map_df = gpd.GeoDataFrame(grid_map_df, geometry='geometry')

#### 연속지적도

In [4]:
cadastral_dic = {'전' : '밭',
                '답' : '논',
                '과' : '과수원',
                '목' : '목장',
                '임' : '임야',
                '광' : '광천지',
                '염' : '염전',
                '대' : '대지',
                '장' : '공장용지',
                '학' : '학교용지',
                '차' : '주차장',
                '주' : '주유소용지',
                '창' : '창고용지',
                '도' : '도로',
                '철' : '철도용지',
                '제' : '제방',
                '천' : '하천',
                '구' : '도랑',
                '유' : '유지',
                '양' : '양어장',
                '수' : '수도용지',
                '공' : '공원용지',
                '체' : '체육용지',
                '원' : '유원지',
                '종' : '종교용지',
                '사' : '사적지',
                '묘' : '묘지',
                '잡' : '잡종지',
                '기타' : '기타'}

In [5]:
# GeoJSON 파일 불러오기
with open('SBJ_2406_002/8.세종시_연속지적도.geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
cadastral_map_df = pd.json_normalize(geojson_data['features'])
cadastral_map_df['geometry'] = cadastral_map_df['geometry.coordinates'].apply(lambda x : make_pol(x))
cadastral_map_df.drop(columns="geometry.coordinates", inplace=True)
# cadastral_map_df 데이터프레임을 GeoDataFrame으로 변환
cadastral_map_df = gpd.GeoDataFrame(cadastral_map_df, geometry='geometry')
# 정규식을 사용하여 'properties.JIBUN'을 지번과 용도로 분리
# 지번은 숫자 또는 숫자-숫자 형식, 용도는 공백 뒤에 오는 한글 문자로 가정
cadastral_map_df['JIBUN'] = cadastral_map_df['properties.JIBUN'].str.extract(r'(\d+-?\d*)')
cadastral_map_df['USAGE'] = cadastral_map_df['properties.JIBUN'].apply(lambda x:x[-1])
# USAGE가 빈 문자열이거나 숫자로만 되어 있는 경우 '기타'로 설정
cadastral_map_df['USAGE'] = cadastral_map_df['USAGE'].apply(lambda x: '기타' if x == '' or x.isdigit() else x)
cadastral_map_df['USAGE'] = cadastral_map_df['USAGE'].apply(lambda x: cadastral_dic.get(x, x))
cadastral_map_df = cadastral_map_df[['JIBUN', 'USAGE', 'geometry']]

#### 건물정보 (국토교통부 - 디지털트윈국토)

In [6]:
# shp to geodataframe convert
shapefile_path = "SBJ_2406_002/(도로명주소)건물_세종/Z_KAIS_TL_SPBD_BULD_36_202406.shp"
sejong_building_gdf = gpd.read_file(shapefile_path, encoding='cp949')
lst = ['LNBR_MNNM', 'LNBR_SLNO', 'BD_MGT_SN', 'BUL_MAN_NO', 'SIG_CD', 'EQB_MAN_SN', 'BULD_MNNM', 'BULD_SLNO', 'BULD_NM', 'BDTYP_CD', 'BUL_DPN_SE', 'GRO_FLO_CO', 'UND_FLO_CO', 'geometry']
sejong_building_gdf = sejong_building_gdf[lst]
sejong_building_gdf = sejong_building_gdf.to_crs(epsg=4326) #EPSG4326 형식으로 변환

In [ ]:
# 맵 객체 생성 및 데이터 로드
sejong_building_map = KeplerGl(height=1000, width=1500)
sejong_building_map.add_data(data=sejong_gdf, name="세종시 행정구역 데이터")
sejong_building_map.add_data(data=sejong_building_gdf, name="세종시 건물 데이터")

# 맵 출력 및 상세설정
sejong_building_map

In [ ]:
# 맵 저장
sejong_building_map.save_to_html(file_name="visualization/세종시 현황/세종시 건물시각화 map.html")

Map saved to visualization/세종시 현황/세종시 건물시각화 map.html!


#### 상가 개폐업 정보

In [5]:
arcade_oc_df = pd.read_csv('SBJ_2406_002/5.세종시_상가개폐업정보.csv')
arcade_oc_df = geo_transform(arcade_oc_df)
# 공간 조인 수행
arcade_oc_df = gpd.sjoin(arcade_oc_df, sejong_gdf, how='inner', op='intersects')
# 신도시 구역 필터링
arcade_oc_df = arcade_oc_df[arcade_oc_df['newtown'] == '신도시']
arcade_oc_df = arcade_oc_df[['service_nm', 'lcpmt_dt', 'rtrcn_dt', 'biz_stts_cd', 'biz_stts_nm',
                            'cls_date', 'tc_strt_dt', 'tc_end_dt', 're_op_dt', 'plc_area',
                            'bplc_nm', 'biz_type','rd_addr', 'lon', 'lat', 'geometry', 'ADM_NM', 'newtown']]

C:\Users\HJK\AppData\Local\Temp\ipykernel_27052\25937912.py:1: DtypeWarning:

Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.

100%|██████████| 43974/43974 [00:00<00:00, 70963.37it/s]
c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWar

In [6]:
arcade_oc_df['biz_stts_nm'].value_counts()

biz_stts_nm
영업/정상             14188
폐업                 8677
취소/말소/만료/정지/중지      364
휴업                   39
Name: count, dtype: int64

In [7]:
# '폐업'인 경우 2020년 이후의 데이터만 필터링
arcade_oc_df['cls_date'] = pd.to_datetime(arcade_oc_df['cls_date'], errors='coerce')
filtered_df = arcade_oc_df[(arcade_oc_df['biz_stts_nm'] == '폐업') & (arcade_oc_df['cls_date'] >= '2020-01-01')]

# '영업/정상'인 데이터프레임과 필터링된 '폐업' 데이터프레임을 결합
normal_df = arcade_oc_df[arcade_oc_df['biz_stts_nm'] == '영업/정상']
filtered_arcade_oc_df = pd.concat([normal_df, filtered_df])

# 행정동별 영업 정상 및 폐업 상태 집계
status_counts = filtered_arcade_oc_df.groupby(['ADM_NM', 'biz_stts_nm']).size().unstack(fill_value=0)

In [8]:
# Extracting year from cls_date for businesses with a closure date
filtered_arcade_oc_df['Year'] = filtered_arcade_oc_df['cls_date'].dt.year

# Setting the year for '영업/정상' businesses to 2023 for calculation purposes (assuming data until 2023)
filtered_arcade_oc_df['Year1'] = 2020
filtered_arcade_oc_df['Year2'] = 2021
filtered_arcade_oc_df['Year3'] = 2022
filtered_arcade_oc_df['Year4'] = 2023

# Counting the number of closed and operating businesses per year and per administrative area
closure_counts = filtered_arcade_oc_df[filtered_arcade_oc_df['biz_stts_nm'] == '폐업'].groupby(['Year', 'ADM_NM']).size().unstack(fill_value=0)
normal_counts1 = filtered_arcade_oc_df[filtered_arcade_oc_df['biz_stts_nm'] == '영업/정상'].groupby(['Year1', 'ADM_NM']).size().unstack(fill_value=0)
normal_counts2 = filtered_arcade_oc_df[filtered_arcade_oc_df['biz_stts_nm'] == '영업/정상'].groupby(['Year2', 'ADM_NM']).size().unstack(fill_value=0)
normal_counts3 = filtered_arcade_oc_df[filtered_arcade_oc_df['biz_stts_nm'] == '영업/정상'].groupby(['Year3', 'ADM_NM']).size().unstack(fill_value=0)
normal_counts4 = filtered_arcade_oc_df[filtered_arcade_oc_df['biz_stts_nm'] == '영업/정상'].groupby(['Year4', 'ADM_NM']).size().unstack(fill_value=0)
normal_counts = pd.concat([normal_counts1, normal_counts2, normal_counts3, normal_counts4])

# Calculating the total number of businesses
total_counts = closure_counts.add(normal_counts, fill_value=0)

# Calculating closure rates
closure_rates = (closure_counts / total_counts) * 100

# Resetting index for proper DataFrame format
closure_rates = closure_rates.reset_index()

# Displaying the closure rates DataFrame
closure_rates = closure_rates.iloc[:4]
closure_rates['Year'] = closure_rates['Year'].astype(int)
closure_rates.to_csv("행정동별_상가폐업률.csv", index=False)

In [9]:
# 폐업률 계산
status_counts['폐업률'] = (status_counts['폐업'] / (status_counts['영업/정상'] + status_counts['폐업'])) * 100
status_counts

biz_stts_nm,영업/정상,폐업,폐업률
ADM_NM,,,
가람동,69,304,81.501340
고운동,1308,534,28.990228
나성동,2073,927,30.900000
누리동,10,1,9.090909
다솜동,0,2,100.000000
다정동,739,347,31.952118
대평동,549,281,33.855422
도담동,916,413,31.075997
반곡동,626,152,19.537275


In [10]:
# 전체 가게건수 계산
status_counts['전체 가게건수'] = status_counts['영업/정상'] + status_counts['폐업']

# 시각화를 위한 데이터프레임 생성 및 전체 가게건수 높은순으로 정렬
plot_df_total = status_counts.reset_index()[['ADM_NM', '전체 가게건수']].sort_values(by='전체 가게건수', ascending=False)

# 시각화
fig_total = px.bar(plot_df_total, x='ADM_NM', y='전체 가게건수', color='전체 가게건수', color_continuous_scale='plasma')
fig_total.show()

In [11]:
# 시각화를 위한 데이터프레임 생성 및 전체 가게건수 높은순으로 정렬
status_counts = status_counts.reset_index()
status_counts_sorted = status_counts.sort_values(by='전체 가게건수', ascending=False)

# 시각화
fig = px.bar(status_counts_sorted, x='ADM_NM', y='폐업률', color='폐업률', color_continuous_scale='plasma')
fig.show()

In [ ]:
# 맵 객체 생성 및 데이터 로드
sejong_arcade_oc_map = KeplerGl(height=1000, width=1500)
sejong_arcade_oc_map.add_data(data=sejong_gdf, name="세종시 행정구역 데이터")
sejong_arcade_oc_map.add_data(data=arcade_oc_df, name="세종시 상가개폐업 데이터")

# 맵 출력 및 상세설정
sejong_arcade_oc_map

In [142]:
# 맵 저장
sejong_arcade_oc_map.save_to_html(file_name="visualization/세종시 현황/세종시 상가개폐업 map.html")

Map saved to visualization/세종시 현황/세종시 상가개폐업 map.html!


In [225]:
filtered_df_rd = filtered_df[~filtered_df['rd_addr'].isnull()]
filtered_df_rd = filtered_df_rd[~filtered_df_rd['rd_addr'].str.contains('*', regex=False)]
filtered_df_rd = filtered_df_rd[~filtered_df_rd['plc_area'].isnull()]

# "~~시 ~~로 숫자" 형식으로 필터링하고 나머지는 rd_addr_ex 열에 저장
def filter_rd_addr(addr):
    match = re.match(r'(.*?시.*?로\s*\d+)', addr)
    if match:
        main_addr = match.group(1)
        extra_addr = addr[len(main_addr):].strip()
        return main_addr, extra_addr
    return addr, ''

filtered_df_rd[['rd_addr', 'rd_addr_ex']] = filtered_df_rd['rd_addr'].apply(lambda x: pd.Series(filter_rd_addr(x)))
filtered_df_rd

,service_nm,lcpmt_dt,rtrcn_dt,biz_stts_cd,biz_stts_nm,cls_date,tc_strt_dt,tc_end_dt,re_op_dt,plc_area,bplc_nm,biz_type,rd_addr,lon,lat,geometry,ADM_NM,newtown,rd_addr_ex
11964,식품자동판매기업,2022-07-05,NaN,3,폐업,2024-04-24,NaN,NaN,NaN,5.88,지 헨즈(g hans),식품자동판매기영업,세종특별자치시 남세종로 160,127.319100,36.486243,POINT (127.31910 36.48624),집현동,신도시,근린생활시설동 1층 9호 (집현동 새나루마을1단지)
23563,일반음식점,2021-11-16,NaN,3,폐업,2024-04-22,NaN,NaN,NaN,97.65,허바허바 세종집현점,중국식,세종특별자치시 집현동로 77,127.325582,36.491039,POINT (127.32558 36.49104),집현동,신도시,124 125호 (집현동)
23596,일반음식점,2021-12-03,NaN,3,폐업,2023-11-23,NaN,NaN,NaN,52.65,강남반찬도시락김밥,한식,세종특별자치시 집현동로 77,127.325582,36.491039,POINT (127.32558 36.49104),집현동,신도시,1층 114호 (집현동)
23751,일반음식점,2022-03-21,NaN,3,폐업,2022-11-01,NaN,NaN,NaN,98.78,오늘의 밥상,한식,세종특별자치시 집현중앙7로 6,127.331219,36.498384,POINT (127.33122 36.49838),집현동,신도시,지식산업센터 2층 B209~B210호 (집현동)
24030,일반음식점,2022-09-05,NaN,3,폐업,2023-10-06,NaN,NaN,NaN,48.29,주식 보리보리,한식,세종특별자치시 집현동로 77,127.325582,36.491039,POINT (127.32558 36.49104),집현동,신도시,1층 124호 (집현동)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27632,즉석판매제조가공업,2021-09-07,NaN,3,폐업,2023-03-07,NaN,NaN,NaN,32.89,팔도만찬 (세종해밀점),즉석판매제조가공업,세종특별자치시 해밀3로 32,127.265646,36.526838,POINT (127.26565 36.52684),해밀동,신도시,해밀마을1단지 지1층 130호 (해밀동 해밀마을1단지)
29296,집단급식소,2020-09-03,NaN,3,폐업,2021-04-02,NaN,NaN,NaN,785.56,해밀중학교,학교,세종특별자치시 해밀2로 6,127.267452,36.527818,POINT (127.26745 36.52782),해밀동,신도시,해밀중학교 (해밀동)
43089,휴게음식점,2020-11-02,NaN,3,폐업,2022-12-30,NaN,NaN,NaN,1.00,씨유 세종마스터힐스2차점,편의점,세종특별자치시 해밀3로 90,127.271004,36.527172,POINT (127.27100 36.52717),해밀동,신도시,상가동 지2층 129호 (해밀동 해밀마을2단지)
43231,휴게음식점,2021-05-31,NaN,3,폐업,2023-11-27,NaN,NaN,NaN,48.11,쁘띠크렘(Petit creme),커피숍,세종특별자치시 해밀3로 32,127.265646,36.526838,POINT (127.26565 36.52684),해밀동,신도시,근린생활시설동 지1층 134호 (해밀동 해밀마을1단지)


In [ ]:
filtered_df_rd

#### 카드매출(격자매핑)

In [12]:
sales_df = pd.read_csv('SBJ_2406_002/9.세종시_카드매출(격자매핑).csv')

# 격자 ID와 격자 폴리곤 각각을 매핑한 딕셔너리 생성
map_dic = dict(zip(grid_map_df['properties.gid'], grid_map_df['geometry']))

# 생성된 map_dic에 filtered_respop의 gid를 매핑하여 'geometry' column 생성
geometry_lst = []
for _, row in sales_df.iterrows():
    geometry_lst.append(map_dic[row['gid']])
sales_df['geometry'] = geometry_lst
sales_df = gpd.GeoDataFrame(sales_df, geometry='geometry')
sales_df = gpd.sjoin(sales_df, sejong_gdf, how='inner', op='intersects')
sales_df = sales_df[sales_df['newtown'] == '신도시']

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning:

The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.



In [90]:
sales_df

,STDR_YM,KBC_BZC_NM_1,KBC_BZC_NM_2,KBC_BZC_NM_3,FRAN_NM,CNS_X,CNS_Y,CARD_SALES,EST_SALES,gid,geometry,index_right,ADM_NM,newtown
1,202301,음식,커피/음료,커피전문점,(주)커피링크,127.301390,36.485671,30000.0,NaN,다바822320,"POLYGON ((127.30128 36.48547, 127.30127 36.486...",1,소담동,신도시
2,202301,서비스,여가/오락서비스,다이어트운동센터,트루밸런스,127.301390,36.485671,0.0,0.0,다바822320,"POLYGON ((127.30128 36.48547, 127.30127 36.486...",1,소담동,신도시
3,202301,서비스,생활편의서비스,인쇄업,서일미디어,127.301390,36.485671,284050.0,7712.0,다바822320,"POLYGON ((127.30128 36.48547, 127.30127 36.486...",1,소담동,신도시
4,202301,기타,기업,기타기업,엘리스스쿼드(ELS),127.301390,36.485671,0.0,0.0,다바822320,"POLYGON ((127.30128 36.48547, 127.30127 36.486...",1,소담동,신도시
5,202301,소매업,의류,일반의류(기타),아더(OTHER),127.301390,36.485671,245600.0,10843.0,다바822320,"POLYGON ((127.30128 36.48547, 127.30127 36.486...",1,소담동,신도시
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24337,202302,음식,한식,일반한식/백반,행복한 밥상,127.297942,36.523969,126000.0,3595.0,다바819362,"POLYGON ((127.29783 36.52333, 127.29783 36.524...",19,누리동,신도시
46932,202303,음식,한식,일반한식/백반,행복한 밥상,127.297942,36.523969,324000.0,3811.0,다바819362,"POLYGON ((127.29783 36.52333, 127.29783 36.524...",19,누리동,신도시
57061,202304,음식,한식,일반한식/백반,행복한 밥상,127.297942,36.523969,90000.0,1473.0,다바819362,"POLYGON ((127.29783 36.52333, 127.29783 36.524...",19,누리동,신도시
100827,202305,음식,한식,일반한식/백반,행복한 밥상,127.297942,36.523969,222000.0,1603.0,다바819362,"POLYGON ((127.29783 36.52333, 127.29783 36.524...",19,누리동,신도시


In [99]:
sales_df_sum = sales_df.groupby(['gid', 'geometry', 'newtown']).agg({'EST_SALES': 'sum'}).reset_index()
sales_df_sum = gpd.GeoDataFrame(sales_df_sum, geometry='geometry')
sales_df_mean = sales_df.groupby(['gid', 'geometry', 'newtown']).agg({'EST_SALES': 'mean'}).reset_index()
sales_df_mean = gpd.GeoDataFrame(sales_df_mean, geometry='geometry')

In [ ]:
# 맵 객체 생성 및 데이터 로드
sejong_sales_map = KeplerGl(height=1000, width=1500)
sejong_sales_map.add_data(data=sejong_gdf, name="세종시 행정구역 데이터")
sejong_sales_map.add_data(data=sales_df_sum, name="세종시 격자 총매출 데이터")
sejong_sales_map.add_data(data=sales_df_mean, name="세종시 격자 평균매출 데이터")

# 맵 출력 및 상세설정
sejong_sales_map

In [104]:
# 맵 저장
sejong_sales_map.save_to_html(file_name="visualization/세종시 현황/세종시 격자별 매출 map.html")

Map saved to visualization/세종시 현황/세종시 격자별 매출 map.html!


In [113]:
sejong_sales_sum = sales_df.groupby(['ADM_NM']).agg({'EST_SALES': 'sum'}).reset_index()
sejong_sales_mean = sales_df.groupby(['ADM_NM']).agg({'EST_SALES': 'mean'}).reset_index()

In [116]:
# 시각화를 위한 데이터프레임 생성 및 전체 가게건수 높은순으로 정렬
plot_df_total = sejong_sales_sum.reset_index().sort_values(by='EST_SALES', ascending=False)

# 시각화
fig_total = px.bar(plot_df_total, x='ADM_NM', y='EST_SALES', color='EST_SALES', color_continuous_scale='plasma')
fig_total.show()

In [119]:
# 시각화를 위한 데이터프레임 생성 및 전체 가게건수 높은순으로 정렬
plot_df_total = sejong_sales_mean.set_index('ADM_NM').reindex(plot_df_total['ADM_NM']).reset_index()

# 시각화
fig_total = px.bar(plot_df_total, x='ADM_NM', y='EST_SALES', color='EST_SALES', color_continuous_scale='plasma')
fig_total.show()

In [121]:
sejong_sales_sum.to_csv("행정동별_총매출.csv", index=False)
sejong_sales_mean.to_csv("행정동별_평균매출.csv", index=False)